In [ ]:
from PIL import Image
import numpy as np
import os
import pickle
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torch.utils.data import Dataset, DataLoader

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
noise_levels = [0,0.1,0.2,0.3]

In [3]:
class CityscapeDataset(Dataset):
    
    def __init__(self, images, labels, noise_level = 0):
#         self.image_dir = image_dir
#         self.image_fns = os.listdir(image_dir)
#         self.label_model = label_model
        self.images = images
        self.labels = labels
        self.noise_level = noise_level
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
#         mean = 0
#         var = 0.1
#         sigma = var**0.5
        if self.noise_level!=0:
            image = image+(self.noise_level*np.random.normal(0, (image.max() - image.min()), image.shape)).astype("uint8") # (mean, sigma, image_shape)
#             gauss = np.random.normal(mean,sigma,image.shape).astype("uint8")
#             image = image+gauss
        image = self.transform(image)
        label = torch.from_numpy(label).long()
        return image, label
            # gaussian nosie
            # sigma could be changed, leave it for now
                # e.g.(image.max() - image.min())/6.
#             print(image.max()) -255
#             print(image.min()) -0
#             print(image.dtype) -unint8 
        
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) # normalize to control the "dynamic range" of activations of different layers
        ])
        return transform_ops(image)

NameError: name 'Dataset' is not defined

A function to transform original data to noisy data

In [ ]:
def get_data_for_one_domain(noise_level,original_dataset,mode = "PCA"):
    pca = PCA(n_components=2)
    
    if noise_level == 0:
        dataset = original_dataset
    else:
        dataset = transform_dataset(original_dataset,noise_level=noise_level)

    flat = np.reshape(dataset,(500, 256*256*3))
    
    return flat

In [ ]:
def transform_dataset(original_dataset,noise_level=0):
    new_dataset = []
    if noise_level == 0:
        print("No transformation needed")
        return original_dataset
    for image in original_dataset:
        if noise_level!=0:
            new_image = image+(noise_level*np.random.normal(0, (image.max() - image.min())/6., image.shape)).astype("uint8") # (mean, sigma, image_shape)
            new_dataset.append(new_image)
    new_dataset = np.array(new_dataset)
    return new_dataset

In [ ]:
input_image_list = [get_data_for_one_domain(noise_levels[i], original_dataset = X_test) for i in range(len(noise_levels))]

In [ ]:
input_image_array = np.array(input_image_list)
input_image_array = input_image_array.reshape((input_image_array.shape)[0]*(input_image_array.shape)[1],(input_image_array.shape)[2])
pca_50 = PCA(n_components = 50)
pca_embedding = pca.fit_transform(input_image_array)

In [ ]:
df_input_full = pd.DataFrame(embedding,columns=["tsne_1","tsne_2"])